In [2]:
from google.colab import drive
drive.mount('/content/drive')

ModuleNotFoundError: No module named 'google'

In [3]:
import pandas as pd
data = pd.read_csv('/content/drive/MyDrive/Research_Dysgraphia/ANN-Orginal.csv')

ModuleNotFoundError: No module named 'pandas'

In [ ]:
data.info()
data.head()

In [ ]:
# Columns to impute with mean
mean_impute_cols = [
    'Age',
    'Weight',
    'OFC',
    'Height',
    'Birth Weight'
]

# Impute with mean where applicable
for col in mean_impute_cols:
    if col in data.columns:  # Check if the column exists in the DataFrame
        mean_value = data[col].mean()
        data[col].fillna(mean_value, inplace=True)

# Impute rest of the columns with mode value
for col in data.columns:
    if col not in mean_impute_cols:  # Only apply to columns not in the mean_impute_cols
        mode_value = data[col].mode().iloc[0]
        data[col].fillna(mode_value, inplace=True)

In [ ]:
import pandas as pd

# Set options to display more rows and/or columns
pd.set_option('display.max_columns', None)  # None means show all columns
pd.set_option('display.max_rows', 100)      # Adjust number of rows to display

# Now print the DataFrame
print(data)

In [ ]:
import pandas as pd
import numpy as np
import random

# Debugging print to check input data type
print(data['Monthly Income'].apply(type))

def calculate_average(income_range):
    income_range = income_range.replace(',', '')  # Remove commas first
    low, high = income_range.split('-')
    return (int(low.strip()) + int(high.strip())) / 2

def impute_monthly_income(value):
    value = str(value).strip()  # Convert to string and remove spaces
    print(f"Processing value: {value}")  # Debugging print to track values
    if '-' in value:
        return calculate_average(value)
    elif '>' in value:
        # Ensure no spaces between '>' and the number
        return random.randint(50001, 100000)  # Generate a random value greater than 50,000
    elif value.lower() == 'n/a':
        return np.nan  # Convert 'N/A' to NaN
    else:
        return np.nan  # Return NaN for any other non-numeric or unhandled format


# Apply the imputation function to the 'Monthly Income' column
data['Monthly Income'] = data['Monthly Income'].apply(impute_monthly_income)

# Display the first few rows of the data to check
data.head()


# Replace 'N/A' values with the mode of the column
mode_value = data['Monthly Income'].dropna().mode()[0]  # Calculate mode excluding NaN
data['Monthly Income'].fillna(mode_value, inplace=True)


# Show the modified DataFrame
print(data.head())  # Display the first few rows to verify changes


In [ ]:
import pandas as pd

# Define the mapping for the 'Mother' and 'Father' columns

education_mapping = {
    'Graduate': 1,
    'A/L': 2,
    'O/L': 3,
    'Primary to O/L': 4,
    'Primary': 5
}

# Apply the mappings

data['Education_Mother'] = data['Education_Mother'].map(education_mapping)
data['Education_Father'] = data['Education_Father'].map(education_mapping)


print(data.head())



In [ ]:
import pandas as pd


# Define the weighted values for each feature

eye_contact_mapping = {
    'Eye Contact-Good': 1,
    'Eye Contact_Fair': 2,
    'Eye Contact_Poor': 3
}
gait_mapping = {
    'Gait_Stable': 1,
    'Gait_Ataxia': 2,
    'Gait_Wide based': 3
}

activity_level_mapping = {
    'Activity Level_Noraml': 1,
    'Activity Level_Overactive': 2,
    'Activity Level_Impulsive': 3
}
attention_mapping = {
    'Attention_Inact ': 1,
    'Attention_Limited': 2,
    'Attention_Impaired': 3
}
behaviour_mapping = {
    'Behaviour_Normal': 1,
    'Behaviour_Uncooperative': 2,
    'Behaviour_Anxious': 3,
    'Behaviour_Disorganized': 4
}
speech_mapping = {
    'Speech_Normal': 1,
    'Speech_Slow': 3,
    'Speech_Fast': 2,
    'Speech_Disfluent': 4,
    'Speech_Mute': 5,
    'Speech_Stammering': 6,
    'Speech_Prominal Reversal': 7
}

# Merge and map the columns
data['Eye Contact'] = data[['Eye Contact-Good', 'Eye Contact_Fair', 'Eye Contact_Poor']].idxmax(axis=1).map(eye_contact_mapping)
data['Gait'] = data[['Gait_Stable', 'Gait_Ataxia', 'Gait_Wide based']].idxmax(axis=1).map(gait_mapping)
data['Activity Level'] = data[['Activity Level_Noraml', 'Activity Level_Overactive', 'Activity Level_Impulsive']].idxmax(axis=1).map(activity_level_mapping)
data['Attention'] = data[['Attention_Inact ', 'Attention_Limited', 'Attention_Impaired']].idxmax(axis=1).map(attention_mapping)
data['Behaviour'] = data[['Behaviour_Normal', 'Behaviour_Uncooperative', 'Behaviour_Anxious', 'Behaviour_Disorganized']].idxmax(axis=1).map(behaviour_mapping)
data['Speech'] = data[['Speech_Normal', 'Speech_Slow', 'Speech_Fast', 'Speech_Disfluent', 'Speech_Mute', 'Speech_Stammering', 'Speech_Prominal Reversal']].idxmax(axis=1).map(speech_mapping)

# Drop the original columns if no longer needed
columns_to_drop = ['Eye Contact-Good', 'Eye Contact_Fair', 'Eye Contact_Poor', 'Gait_Stable', 'Gait_Ataxia', 'Gait_Wide based', 'Activity Level_Noraml', 'Activity Level_Overactive', 'Activity Level_Impulsive', 'Attention_Inact ', 'Attention_Limited', 'Attention_Impaired', 'Behaviour_Normal', 'Behaviour_Uncooperative', 'Behaviour_Anxious', 'Behaviour_Disorganized', 'Speech_Normal', 'Speech_Slow', 'Speech_Fast', 'Speech_Disfluent', 'Speech_Mute', 'Speech_Stammering', 'Speech_Prominal Reversal']
data.drop(columns=columns_to_drop, inplace=True)

# Ensure severity is the last column
severity = data.pop('severity')
data['severity'] = severity

print(data.head())





In [ ]:
import pandas as pd

# Define the weighted values for each feature
expressive_language_mapping = {
    'Expressive Language_Appropirate': 1,
    'Expressive Language_Immature use of language': 2,
    'Expressive Language_Primarily uses gestures': 3
}


reading_mapping = {
    'Reading_Above Average': 1,
    'Reading_Average': 3,
    'Reading_Below Average': 5
}

writing_mapping = {
    'Writing_Above Average': 1,
    'Writing_Average': 3,
    'Writing_Below Average': 5
}

mathematics_mapping = {
    'Mathematics_Above Average': 1,
    'Mathematics_Average': 3,
    'Mathematics_Below Average': 5
}

# Functional skills mappings
feeding_mapping = {
    'Feeding_Independent': 1,
    'Feeding_Needs support': 2,
    'Feeding_Dependent': 3
}

bathing_mapping = {
    'Bathing_Independent': 1,
    'Bathing_Needs support': 2,
    'Bathing_Dependent': 3
}

mobility_mapping = {
    'Mobility_Independent': 1,
    'Mobility_Needs support': 2,
    'Mobility_Dependent': 3
}

dressing_mapping = {
    'Dressing_Independent': 1,
    'Dressing_Needs support': 2,
    'Dressing_Dependent': 3
}

continence_mapping = {
    'Continence_Independent': 1,
    'Continence_Needs support': 2,
    'Continence_Dependent': 3
}

# Other Skills mappings
self_care_mapping = {
    'Self-care_Satisafactory': 1,
    'Self-care_Unsatisfactory': 2
}

home_living_mapping = {
    'Home-living_Satisafactory': 1,
    'Home-living_Unsatisfactory': 2
}

social_skills_mapping = {
    'Social-skills_Satisafactory': 1,
    'Social-skills_Unsatisfactory': 2
}

safety_mapping = {
    'Safety_Satisafactory': 1,
    'Safety_Unsatisfactory': 2
}

leisure_activities_mapping = {
    'Leisure Activites_Satisafactory': 1,
    'Leisure Activites_Unsatisfactory': 2
}

# Assuming your data DataFrame contains all these columns, the below code merges and maps them

# Mapping Expressive Language, IQ, Reading, Writing, Mathematics and others
data['Expressive Language'] = data[['Expressive Language_Appropirate', 'Expressive Language_Immature use of language', 'Expressive Language_Primarily uses gestures']].idxmax(axis=1).map(expressive_language_mapping)
data['Reading'] = data[['Reading_Above Average', 'Reading_Average', 'Reading_Below Average']].idxmax(axis=1).map(reading_mapping)
data['Writing'] = data[['Writing_Above Average', 'Writing_Average', 'Writing_Below Average']].idxmax(axis=1).map(writing_mapping)
data['Mathematics'] = data[['Mathematics_Above Average', 'Mathematics_Average', 'Mathematics_Below Average']].idxmax(axis=1).map(mathematics_mapping)

# Mapping functional skills
data['Feeding'] = data[['Feeding_Independent', 'Feeding_Needs support', 'Feeding_Dependent']].idxmax(axis=1).map(feeding_mapping)
data['Bathing'] = data[['Bathing_Independent', 'Bathing_Needs support', 'Bathing_Dependent']].idxmax(axis=1).map(bathing_mapping)
data['Mobility'] = data[['Mobility_Independent', 'Mobility_Needs support', 'Mobility_Dependent']].idxmax(axis=1).map(mobility_mapping)
data['Dressing'] = data[['Dressing_Independent', 'Dressing_Needs support', 'Dressing_Dependent']].idxmax(axis=1).map(dressing_mapping)
data['Continence'] = data[['Continence_Independent', 'Continence_Needs support', 'Continence_Dependent']].idxmax(axis=1).map(continence_mapping)

# Mapping other skills
data['Self-care'] = data[['Self-care_Satisafactory', 'Self-care_Unsatisfactory']].idxmax(axis=1).map(self_care_mapping)
data['Home-living'] = data[['Home-living_Satisafactory', 'Home-living_Unsatisfactory']].idxmax(axis=1).map(home_living_mapping)
data['Social-skills'] = data[['Social-skills_Satisafactory', 'Social-skills_Unsatisfactory']].idxmax(axis=1).map(social_skills_mapping)
data['Safety'] = data[['Safety_Satisafactory', 'Safety_Unsatisfactory']].idxmax(axis=1).map(safety_mapping)
data['Leisure Activites'] = data[['Leisure Activites_Satisafactory','Leisure Activites_Unsatisfactory']].idxmax(axis=1).map(leisure_activities_mapping)

# Drop the original columns if no longer needed
columns_to_drop = ['Expressive Language_Appropirate', 'Expressive Language_Immature use of language', 'Expressive Language_Primarily uses gestures',
                   'Reading_Above Average', 'Reading_Average', 'Reading_Below Average',
                   'Writing_Above Average', 'Writing_Average', 'Writing_Below Average',
                   'Mathematics_Above Average', 'Mathematics_Average', 'Mathematics_Below Average',
                   'Feeding_Independent', 'Feeding_Needs support', 'Feeding_Dependent',
                   'Bathing_Independent', 'Bathing_Needs support', 'Bathing_Dependent',
                   'Mobility_Independent', 'Mobility_Needs support', 'Mobility_Dependent',
                   'Dressing_Independent', 'Dressing_Needs support', 'Dressing_Dependent',
                   'Continence_Independent', 'Continence_Needs support', 'Continence_Dependent',
                   'Self-care_Satisafactory', 'Self-care_Unsatisfactory',
                   'Home-living_Satisafactory', 'Home-living_Unsatisfactory',
                   'Social-skills_Satisafactory', 'Social-skills_Unsatisfactory',
                   'Safety_Satisafactory', 'Safety_Unsatisfactory','Leisure Activites_Satisafactory','Leisure Activites_Unsatisfactory'
]


data.drop(columns=columns_to_drop, inplace=True)

# Ensure severity is the last column
severity = data.pop('severity')
data['severity'] = severity

print(data.info())
print(data.head())

# Save the modified DataFrame
data.to_csv('/content/drive/MyDrive/Research_Dysgraphia/ANN_After_Feature_Engineering.csv', index=False)


In [ ]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

# Load the data
file_path = '/content/drive/MyDrive/Research_Dysgraphia/ANN_After_Feature_Engineering.csv'
data = pd.read_csv(file_path)


# Initialize MinMaxScaler
scaler = MinMaxScaler()

# Select columns to scale (excluding 'Child ID' and already binary columns)
columns_to_scale = data.columns.drop('Child ID')  # Adjust this to exclude any other non-continuous columns

# Identify columns that are not already 0 or 1
columns_to_scale = [col for col in columns_to_scale if not all(data[col].isin([0, 1]))]

# Apply the scaler to these columns
data[columns_to_scale] = scaler.fit_transform(data[columns_to_scale])

# Round the scaled data to two decimal places
data[columns_to_scale] = data[columns_to_scale].round(2)

# Save the normalized and rounded data
normalized_file_path = '/content/drive/MyDrive/Research_Dysgraphia/ANN_Normalized_data.csv'
data.to_csv(normalized_file_path, index=False)

print(f"Normalized data saved to {normalized_file_path}")


In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Load the normalized data
normalized_file_path = '/content/drive/MyDrive/Research_Dysgraphia/ANN_Normalized_data.csv'
print(f"Loading data from {normalized_file_path}")
data = pd.read_csv(normalized_file_path)
print("Data loaded successfully. Data shape:", data.shape)

# Drop the 'Child ID' column as it is not relevant for the correlation analysis
print("Dropping 'Child ID' column")
data = data.drop('Child ID', axis=1)
print("'Child ID' column dropped. Data shape:", data.shape)

# Calculate the correlation matrix
print("Calculating correlation matrix")
corr_matrix = data.corr()
#spearman_corr = data.corr(method='spearman')
print("Correlation matrix calculated successfully")

# Extract correlation with 'Severity' specifically
print("Extracting correlation with 'Severity'")
severity_corr = corr_matrix[['severity']]
print("Correlation with 'Severity' extracted. Shape:", severity_corr.shape)

# Split the features into batches of 20 for better visualization
num_features = len(severity_corr)
features_per_batch = 20
print(f"Total features: {num_features}, Features per batch: {features_per_batch}")

# Loop through the features in batches of 20
for i in range(0, num_features, features_per_batch):
    batch = severity_corr.iloc[i:i + features_per_batch]
    print(f"Processing batch {i // features_per_batch + 1} with features {i} to {i + len(batch) - 1}")

    # Set up the matplotlib figure
    plt.figure(figsize=(4, len(batch) * 0.5))  # Adjust the figure size to fit the batch size

    # Draw the heatmap with the mask and correct aspect ratio
    sns.heatmap(batch, annot=True, fmt=".2f", cmap='coolwarm', vmin=-1, vmax=1,
                square=False, linewidths=.5, cbar_kws={"shrink": .8})

    # Adjust the plot to make sure all is visible
    plt.xticks(rotation=0)
    plt.yticks(rotation=0, ha='right')
    plt.title(f'Correlation with Severity - Batch {i // features_per_batch + 1}')

    # Show the plot
    print(f"Displaying heatmap for batch {i // features_per_batch + 1}")
    plt.show()

In [ ]:
import pandas as pd
import numpy as np

# Load the data from the specified path
file_path = '/content/drive/MyDrive/Research_Dysgraphia/ANN_Feature Selection.csv'
data = pd.read_csv(file_path)

# Convert 'severity' to categorical bins
bins = [-np.inf, 0.75, np.inf]
labels = ['Low', 'High']
data['severity'] = pd.cut(data['severity'], bins=bins, labels=labels, right=True)

# Save the modified data to the specified path
modified_file_path = '/content/drive/MyDrive/Research_Dysgraphia/ANN_modified_data.csv'
data.to_csv(modified_file_path, index=False)

print("Data processing completed and saved to:", modified_file_path)



In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Load the modified data from the specified path
modified_file_path = '/content/drive/MyDrive/Research_Dysgraphia/ANN_modified_data.csv'
data = pd.read_csv(modified_file_path)

# Split the data into training and testing sets
train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)

# Save the training and testing datasets to specified paths
train_file_path = '/content/drive/MyDrive/Research_Dysgraphia/ANN_train_data.csv'
test_file_path = '/content/drive/MyDrive/Research_Dysgraphia/ANN_test_data.csv'

train_data.to_csv(train_file_path, index=False)
test_data.to_csv(test_file_path, index=False)

print("Data split completed and saved to:")
print("Training data:", train_file_path)
print("Testing data:", test_file_path)


# RandoM Forest-SMOTE Before Cross Validation
**bold text**

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score
import numpy as np
import joblib

# Load the modified data from the specified path
modified_file_path = '/content/drive/MyDrive/Research_Dysgraphia/ANN_modified_data.csv'
data = pd.read_csv(modified_file_path)

# Split the data into training and testing sets
train_data, test_data = train_test_split(data, test_size=0.2, random_state=42, stratify=data.iloc[:, -1])

# Save the training and testing datasets to specified paths
train_file_path = '/content/drive/MyDrive/Research_Dysgraphia/ANN_train_data.csv'
test_file_path = '/content/drive/MyDrive/Research_Dysgraphia/ANN_test_data.csv'

train_data.to_csv(train_file_path, index=False)
test_data.to_csv(test_file_path, index=False)

print("Data split completed and saved to:")
print("Training data:", train_file_path)
print("Testing data:", test_file_path)

# Assume the last column is the target variable, and the rest are features
X = train_data.iloc[:, :-1]  # Features
y = train_data.iloc[:, -1]   # Target

# Split dataset into features and target for training
X_train = X
y_train = y

# Standardize the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)

# Apply SMOTE to handle the data imbalance
smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

# Display the shape of the resampled dataset
print("Before SMOTE:", X_train.shape, y_train.value_counts())
print("After SMOTE:", X_train_resampled.shape, pd.Series(y_train_resampled).value_counts())

# Train a Random Forest Classifier with k-fold cross-validation
rfc = RandomForestClassifier(random_state=42, class_weight='balanced')

# Define Stratified K-Fold Cross Validator
kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
cross_val_scores = cross_val_score(rfc, X_train_resampled, y_train_resampled, cv=kf, scoring='accuracy')

# Display cross-validation results
print("Cross-Validation Accuracy Scores:", cross_val_scores)
print("Mean Cross-Validation Accuracy:", np.mean(cross_val_scores))

# Train on the entire resampled training set
rfc.fit(X_train_resampled, y_train_resampled)

# Load the testing data
X_test = test_data.iloc[:, :-1]  # Features
y_test = test_data.iloc[:, -1]   # Target

# Standardize the test features
X_test = scaler.transform(X_test)

# Make predictions
y_pred = rfc.predict(X_test)

# Evaluate the model
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))

# Save the model to a file
joblib.dump(rfc, '/content/drive/MyDrive/Research_Dysgraphia/random_forest_pipeline.pkl')


# Random Forest-SMOTE during cross-validation

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score
from imblearn.pipeline import make_pipeline
import numpy as np

# Load dataset
modified_file_path = '/content/drive/MyDrive/Research_Dysgraphia/ANN_modified_data.csv'
data = pd.read_csv(modified_file_path)

# Assume the last column is the target variable, and the rest are features
X = data.iloc[:, :-1]  # Features
y = data.iloc[:, -1]   # Target

# Split dataset into training and testing sets
X_train_full, X_test, y_train_full, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# Define the pipeline
pipeline = make_pipeline(
    StandardScaler(),
    SMOTE(random_state=42),
    RandomForestClassifier(random_state=42, class_weight='balanced')
)

# Define Stratified K-Fold Cross Validator
kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Perform cross-validation
cross_val_scores = cross_val_score(
    pipeline, X_train_full, y_train_full, cv=kf, scoring='accuracy'
)

# Display cross-validation results
print("Cross-Validation Accuracy Scores:", cross_val_scores)
print("Mean Cross-Validation Accuracy:", np.mean(cross_val_scores))

# Fit the pipeline on the entire training data
pipeline.fit(X_train_full, y_train_full)

# Make predictions on the test set
y_pred = pipeline.predict(X_test)

# Evaluate the model
print("Accuracy on Test Set:", accuracy_score(y_test, y_pred))
print("Classification Report on Test Set:\n", classification_report(y_test, y_pred))

# Save the model to an HDF5 file
import joblib

# Save the pipeline
joblib.dump(pipeline, '/content/drive/MyDrive/Research_Dysgraphia/random_forest_pipeline.pkl')


# SVM

# SMOTE before the cross validation

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score
import numpy as np
import joblib

# Load the modified data from the specified path
modified_file_path = '/content/drive/MyDrive/Research_Dysgraphia/ANN_modified_data.csv'
data = pd.read_csv(modified_file_path)

# Split the data into training and testing sets
train_data, test_data = train_test_split(data, test_size=0.2, random_state=42, stratify=data.iloc[:, -1])

# Save the training and testing datasets to specified paths
train_file_path = '/content/drive/MyDrive/Research_Dysgraphia/ANN_train_data.csv'
test_file_path = '/content/drive/MyDrive/Research_Dysgraphia/ANN_test_data.csv'

train_data.to_csv(train_file_path, index=False)
test_data.to_csv(test_file_path, index=False)

print("Data split completed and saved to:")
print("Training data:", train_file_path)
print("Testing data:", test_file_path)

# Assume the last column is the target variable, and the rest are features
X = train_data.iloc[:, :-1]  # Features
y = train_data.iloc[:, -1]   # Target

# Split dataset into features and target for training
X_train = X
y_train = y

# Standardize the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)

# Apply SMOTE to handle the data imbalance
smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

# Display the shape of the resampled dataset
print("Before SMOTE:", X_train.shape, y_train.value_counts())
print("After SMOTE:", X_train_resampled.shape, pd.Series(y_train_resampled).value_counts())

# Train an SVM with k-fold cross-validation
svm = SVC(kernel='rbf', class_weight='balanced', random_state=42)

# Define Stratified K-Fold Cross Validator
kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
cross_val_scores = cross_val_score(svm, X_train_resampled, y_train_resampled, cv=kf, scoring='accuracy')

# Display cross-validation results
print("Cross-Validation Accuracy Scores:", cross_val_scores)
print("Mean Cross-Validation Accuracy:", np.mean(cross_val_scores))

# Train on the entire resampled training set
svm.fit(X_train_resampled, y_train_resampled)

# Load the testing data
X_test = test_data.iloc[:, :-1]  # Features
y_test = test_data.iloc[:, -1]   # Target

# Standardize the test features
X_test = scaler.transform(X_test)

# Make predictions
y_pred = svm.predict(X_test)

# Evaluate the model
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))

# Save the model to a file
joblib.dump(svm, '/content/drive/MyDrive/Research_Dysgraphia/svm_pipeline.pkl')


# Testing

In [ ]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, accuracy_score
import joblib

# Load the testing data from the specified path
test_file_path = '/content/drive/MyDrive/Research_Dysgraphia/ANN_test_data.csv'
test_data = pd.read_csv(test_file_path)

# Load the saved model
model_path = '/content/drive/MyDrive/Research_Dysgraphia/svm_pipeline.pkl'
svm = joblib.load(model_path)

# Assume the last column is the target variable, and the rest are features
X_test = test_data.iloc[:, :-1]  # Features
y_test = test_data.iloc[:, -1]   # Target

# Standardize the test features using the same parameters from training
scaler = StandardScaler()
X_train_file_path = '/content/drive/MyDrive/Research_Dysgraphia/ANN_train_data.csv'
train_data = pd.read_csv(X_train_file_path)
X_train = train_data.iloc[:, :-1]
scaler.fit(X_train)
X_test = scaler.transform(X_test)

# Make predictions
y_pred = svm.predict(X_test)

# Evaluate the model
print("Testing Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))


# SMOTE During the cross validation

In [ ]:
import pandas as pd
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score
from imblearn.pipeline import make_pipeline
import numpy as np
import joblib

# Load training dataset
train_file_path = '/content/drive/MyDrive/Research_Dysgraphia/ANN_train_data.csv'
data = pd.read_csv(train_file_path)

# Assume the last column is the target variable, and the rest are features
X_train_full = data.iloc[:, :-1]  # Features
y_train_full = data.iloc[:, -1]   # Target

# Define the pipeline with reduced SMOTE sampling strategy
pipeline = make_pipeline(
    StandardScaler(),
    SMOTE(random_state=42, sampling_strategy=0.5),  # Reduce SMOTE over-sampling
    SVC(kernel='rbf', class_weight='balanced', random_state=42, C=0.5)  # Increase regularization strength
)

# Define Stratified K-Fold Cross Validator
kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Initialize lists to store fold-wise metrics
fold_var = 1
accuracies = []

# Perform K-Fold Cross-Validation manually to track fold-wise metrics
for train_index, val_index in kf.split(X_train_full, y_train_full):
    # Split the data into training and validation sets for the current fold
    X_train, X_val = X_train_full.iloc[train_index], X_train_full.iloc[val_index]
    y_train, y_val = y_train_full.iloc[train_index], y_train_full.iloc[val_index]

    # Fit the pipeline on the training data of the current fold
    pipeline.fit(X_train, y_train)

    # Make predictions on the validation set
    y_val_pred = pipeline.predict(X_val)

    # Calculate accuracy for the current fold
    accuracy = accuracy_score(y_val, y_val_pred)
    accuracies.append(accuracy)

    # Display fold-wise metrics
    print(f"Fold {fold_var} - Accuracy: {accuracy}")
    print(f"Fold {fold_var} - Classification Report:\n{classification_report(y_val, y_val_pred)}")

    # Save the pipeline for the current fold
    joblib.dump(pipeline, f'/content/drive/MyDrive/Research_Dysgraphia/svm_pipeline_fold{fold_var}.pkl')

    # Increment the fold counter
    fold_var += 1

# Display cross-validation results
print("Cross-Validation Accuracy Scores:", accuracies)
print("Mean Cross-Validation Accuracy:", np.mean(accuracies))

# Fit the pipeline on the entire training data
pipeline.fit(X_train_full, y_train_full)

# Make predictions on the training set
y_pred = pipeline.predict(X_train_full)

# Evaluate the model
print("Classification Report on Training Set:\n", classification_report(y_train_full, y_pred))

# Save the final pipeline to a file
joblib.dump(pipeline, '/content/drive/MyDrive/Research_Dysgraphia/svm_pipeline_final.pkl')


# Testing

In [ ]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, accuracy_score
import joblib

# Load testing dataset
test_file_path = '/content/drive/MyDrive/Research_Dysgraphia/ANN_test_data.csv'
test_data = pd.read_csv(test_file_path)

# Assume the last column is the target variable, and the rest are features
X_test = test_data.iloc[:, :-1]  # Features
y_test = test_data.iloc[:, -1]   # Target

# Load the trained pipeline
pipeline = joblib.load(f'/content/drive/MyDrive/Research_Dysgraphia/svm_pipeline_fold{3}.pkl')

# Standardize the test data using the scaler from the trained pipeline
scaler = pipeline.named_steps['standardscaler']
X_test_scaled = scaler.transform(X_test)

# Make predictions on the testing set
y_test_pred = pipeline.named_steps['svc'].predict(X_test_scaled)

# Evaluate the model on testing set
print("Accuracy on Test Set:", accuracy_score(y_test, y_test_pred))
print("Classification Report on Test Set:\n", classification_report(y_test, y_test_pred))


In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Cross-validation accuracies from the previous code
accuracies = [0.85, 0.88, 0.84, 0.86, 0.87]  # Replace with your accuracies list if different

# Assuming these are similar to training and validation metrics over epochs
val_accuracies = [0.83, 0.87, 0.82, 0.85, 0.86]  # Example validation accuracies
train_losses = [0.4, 0.35, 0.3, 0.28, 0.25]  # Example training losses
val_losses = [0.42, 0.37, 0.33, 0.3, 0.27]  # Example validation losses

# Plotting Accuracy Over Folds
plt.figure(figsize=(12, 4))

# Accuracy Plot
plt.subplot(1, 2, 1)
plt.plot(range(1, len(accuracies) + 1), accuracies, marker='o', linestyle='-', color='b', label='Training Accuracy')
plt.plot(range(1, len(val_accuracies) + 1), val_accuracies, marker='o', linestyle='--', color='g', label='Validation Accuracy')
plt.xlabel('Fold Number')
plt.ylabel('Accuracy')
plt.title('Accuracy Over Folds')
plt.xticks(range(1, len(accuracies) + 1))
plt.ylim([0, 1])
plt.grid(True)
plt.legend()

# Loss Plot
plt.subplot(1, 2, 2)
plt.plot(range(1, len(train_losses) + 1), train_losses, marker='o', linestyle='-', color='b', label='Training Loss')
plt.plot(range(1, len(val_losses) + 1), val_losses, marker='o', linestyle='--', color='g', label='Validation Loss')
plt.xlabel('Fold Number')
plt.ylabel('Loss')
plt.title('Loss Over Folds')
plt.xticks(range(1, len(train_losses) + 1))
plt.grid(True)
plt.legend()

plt.show()


In [ ]:
import pandas as pd
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import classification_report, accuracy_score
from imblearn.pipeline import make_pipeline
import numpy as np
import joblib

# Load training dataset
train_file_path = '/content/drive/MyDrive/Research_Dysgraphia/ANN_train_data.csv'
data = pd.read_csv(train_file_path)

# Assume the last column is the target variable, and the rest are features
X_train_full = data.iloc[:, :-1]  # Features
y_train_full = data.iloc[:, -1]   # Target

# Define the pipeline with reduced SMOTE sampling strategy
pipeline = make_pipeline(
    StandardScaler(),
    SMOTE(random_state=42, sampling_strategy=0.5),  # Reduce SMOTE over-sampling
    GradientBoostingClassifier(random_state=42, n_estimators=50, learning_rate=0.05, max_depth=2)  # Reduced complexity to prevent overfitting
)

# Define Stratified K-Fold Cross Validator
kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Initialize lists to store fold-wise metrics
fold_var = 1
accuracies = []

# Perform K-Fold Cross-Validation manually to track fold-wise metrics
for train_index, val_index in kf.split(X_train_full, y_train_full):
    # Split the data into training and validation sets for the current fold
    X_train, X_val = X_train_full.iloc[train_index], X_train_full.iloc[val_index]
    y_train, y_val = y_train_full.iloc[train_index], y_train_full.iloc[val_index]

    # Fit the pipeline on the training data of the current fold
    pipeline.fit(X_train, y_train)

    # Make predictions on the validation set
    y_val_pred = pipeline.predict(X_val)

    # Calculate accuracy for the current fold
    accuracy = accuracy_score(y_val, y_val_pred)
    accuracies.append(accuracy)

    # Display fold-wise metrics
    print(f"Fold {fold_var} - Accuracy: {accuracy}")
    print(f"Fold {fold_var} - Classification Report:\n{classification_report(y_val, y_val_pred)}")

    # Save the pipeline for the current fold
    joblib.dump(pipeline, f'/content/drive/MyDrive/Research_Dysgraphia/gb_pipeline_fold{fold_var}.pkl')

    # Increment the fold counter
    fold_var += 1

# Display cross-validation results
print("Cross-Validation Accuracy Scores:", accuracies)
print("Mean Cross-Validation Accuracy:", np.mean(accuracies))

# Fit the pipeline on the entire training data
pipeline.fit(X_train_full, y_train_full)

# Make predictions on the training set
y_pred = pipeline.predict(X_train_full)

# Evaluate the model
print("Classification Report on Training Set:\n", classification_report(y_train_full, y_pred))

# Save the final pipeline to a file
joblib.dump(pipeline, '/content/drive/MyDrive/Research_Dysgraphia/gb_pipeline_final.pkl')



In [ ]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, accuracy_score
from sklearn.ensemble import GradientBoostingClassifier
import joblib

# Load testing dataset
test_file_path = '/content/drive/MyDrive/Research_Dysgraphia/ANN_test_data.csv'
test_data = pd.read_csv(test_file_path)

# Assume the last column is the target variable, and the rest are features
X_test = test_data.iloc[:, :-1]  # Features
y_test = test_data.iloc[:, -1]   # Target

# Load the trained pipeline
pipeline = joblib.load(f'/content/drive/MyDrive/Research_Dysgraphia/gb_pipeline_fold{1}.pkl')

# Standardize the test data using the scaler from the trained pipeline
scaler = pipeline.named_steps['standardscaler']
X_test_scaled = scaler.transform(X_test)

# Make predictions on the testing set
y_test_pred = pipeline.named_steps['gradientboostingclassifier'].predict(X_test_scaled)

# Evaluate the model on testing set
print("Accuracy on Test Set:", accuracy_score(y_test, y_test_pred))
print("Classification Report on Test Set:\n", classification_report(y_test, y_test_pred))


# Multi Layer Perceptron

## Using SMOTE Inside Each Cross-Validation Fold


Training Dataset


In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import StratifiedKFold, GridSearchCV
from imblearn.over_sampling import ADASYN
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report, accuracy_score
from sklearn.preprocessing import MinMaxScaler

# Load your data
file_path = '/content/drive/MyDrive/Research_Dysgraphia/ANN_modified_data.csv'
data = pd.read_csv(file_path)
X = data.drop('severity', axis=1)
y = data['severity']

# Normalize features
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)

# Define cross-validator and hyperparameter grid for MLPClassifier
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

param_grid = {
    'hidden_layer_sizes': [(10,), (30,), (30, 10)],  # Smaller architectures
    'activation': ['relu', 'tanh'],
    'solver': ['adam', 'lbfgs'],
    'alpha': [0.01, 0.1, 1.0, 10.0],  # Stronger regularization values
    'learning_rate': ['constant', 'adaptive'],
    'max_iter': [100, 200],
}


# Initialize the MLP classifier
mlp = MLPClassifier(hidden_layer_sizes=(100, 50), max_iter=1000, early_stopping=True, random_state=42)

# Set up GridSearchCV
grid_search = GridSearchCV(estimator=mlp, param_grid=param_grid, cv=skf, scoring='accuracy', n_jobs=-1)

# Lists to store results
metrics_list = []

# Perform cross-validation
fold = 0
for train_index, test_index in skf.split(X_scaled, y):
    fold += 1
    X_train_fold, X_test_fold = X_scaled[train_index], X_scaled[test_index]
    y_train_fold, y_test_fold = y[train_index], y[test_index]

    # Apply ADASYN (SMOTE alternative) on each fold training data
    smote = ADASYN(sampling_strategy='auto', random_state=42)
    X_train_fold_smote, y_train_fold_smote = smote.fit_resample(X_train_fold, y_train_fold)

    # Perform grid search to find the best hyperparameters
    grid_search.fit(X_train_fold_smote, y_train_fold_smote)

    # Train the classifier with the best hyperparameters found
    best_classifier = grid_search.best_estimator_

    # Predict on the training fold (training accuracy)
    y_train_pred = best_classifier.predict(X_train_fold_smote)

    # Calculate training accuracy and other metrics
    accuracy_train = accuracy_score(y_train_fold_smote, y_train_pred)
    report_train = classification_report(y_train_fold_smote, y_train_pred, target_names=['High', 'Low'], output_dict=True)

    # Store metrics for each fold
    metrics = {
        'Fold': fold,
        'Training Accuracy': accuracy_train,
        'High Precision': report_train['High']['precision'],
        'High Recall': report_train['High']['recall'],
        'High F1': report_train['High']['f1-score'],
        'Low Precision': report_train['Low']['precision'],
        'Low Recall': report_train['Low']['recall'],
        'Low F1': report_train['Low']['f1-score'],
    }
    metrics_list.append(metrics)

    # Print training metrics for the current fold
    print(f"Training Metrics for Fold {fold}:")
    print(pd.DataFrame([metrics]))

    # Print the best hyperparameters for the current fold
    print(f"Best hyperparameters for Fold {fold}: {grid_search.best_params_}")

# Summarize training metrics across all folds
all_metrics_df = pd.DataFrame(metrics_list)
print("\nSummary of Training Metrics Across All Folds:")
print(all_metrics_df.mean())


Test Dataset

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import StratifiedKFold, GridSearchCV, train_test_split
from imblearn.over_sampling import ADASYN
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report, accuracy_score
from sklearn.preprocessing import MinMaxScaler

# Load your data
file_path = '/content/drive/MyDrive/Research_Dysgraphia/ANN_modified_data.csv'
data = pd.read_csv(file_path)
X = data.drop('severity', axis=1)
y = data['severity']

# Step 1: Split the data into training (80%) and test (20%)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Step 2: Normalize the features
scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Step 3: Apply ADASYN (SMOTE alternative) to the training data only
smote = ADASYN(sampling_strategy='auto', random_state=42)
X_train_smote, y_train_smote = smote.fit_resample(X_train_scaled, y_train)

# Step 4: Define cross-validator and hyperparameter grid for MLPClassifier
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

param_grid = {
    'hidden_layer_sizes': [(50,), (100,), (50, 30), (100, 50)],  # Reduce model complexity
    'activation': ['relu', 'tanh'],  # Activation functions
    'solver': ['adam', 'lbfgs'],  # Solvers for weight optimization
    'alpha': [0.0001, 0.001, 0.01, 0.1, 1.0],  # Increase regularization values
    'learning_rate': ['constant', 'adaptive'],  # Learning rate strategies
    'max_iter': [100, 200, 300],  # Reduce the maximum number of iterations
}

# Initialize the MLP classifier with early stopping
mlp = MLPClassifier(hidden_layer_sizes=(100, 50), max_iter=1000, early_stopping=True, random_state=42)

# Set up GridSearchCV
grid_search = GridSearchCV(estimator=mlp, param_grid=param_grid, cv=skf, scoring='accuracy', n_jobs=-1)

# Step 5: Perform grid search cross-validation on the training set with SMOTE
grid_search.fit(X_train_smote, y_train_smote)

# Step 6: After cross-validation, get the best model
best_classifier = grid_search.best_estimator_

# Step 7: Evaluate the best model on the unseen test data
y_test_pred = best_classifier.predict(X_test_scaled)

# Calculate accuracy and other metrics on the test set
accuracy_test = accuracy_score(y_test, y_test_pred)
report_test = classification_report(y_test, y_test_pred, target_names=['High', 'Low'])

# Print the final test accuracy and classification report
print(f"\nTest Accuracy on the unseen test data: {accuracy_test}")
print("Classification Report on the test data:")
print(report_test)


# Applying SMOTE Before Cross-Validation


Train the model


In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, StratifiedKFold
from imblearn.over_sampling import SMOTE
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report, accuracy_score
from sklearn.preprocessing import MinMaxScaler

# Load your data
file_path = '/content/drive/MyDrive/Research_Dysgraphia/ANN_modified_data.csv'
data = pd.read_csv(file_path)
X = data.drop('severity', axis=1)
y = data['severity']

# Split the data into training and test sets (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Normalize the features for both training and test sets
scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Apply SMOTE to the training set (not the test set)
smote = SMOTE(sampling_strategy='auto', random_state=42)
X_train_smote, y_train_smote = smote.fit_resample(X_train_scaled, y_train)

# Option 1: Use cross-validation on the training set
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
classifier = MLPClassifier(hidden_layer_sizes=(100, 50), max_iter=300, activation='relu', random_state=42)

# Lists to store results
metrics_list = []

# Perform cross-validation on the training data (with SMOTE)
fold = 0
for train_index, test_index in skf.split(X_train_smote, y_train_smote):
    fold += 1
    X_train_fold, X_val_fold = X_train_smote[train_index], X_train_smote[test_index]
    y_train_fold, y_val_fold = y_train_smote[train_index], y_train_smote[test_index]

    # Train classifier on the training fold
    classifier.fit(X_train_fold, y_train_fold)

    # Predict on the validation fold
    y_pred_val = classifier.predict(X_val_fold)

    # Calculate metrics on the validation fold
    accuracy = accuracy_score(y_val_fold, y_pred_val)
    report = classification_report(y_val_fold, y_pred_val, target_names=['High', 'Low'], output_dict=True)

    # Store metrics for each fold
    metrics = {
        'Fold': fold,
        'Overall Accuracy': accuracy,
        'High Precision': report['High']['precision'],
        'High Recall': report['High']['recall'],
        'High F1': report['High']['f1-score'],
        'Low Precision': report['Low']['precision'],
        'Low Recall': report['Low']['recall'],
        'Low F1': report['Low']['f1-score'],
    }
    metrics_list.append(metrics)

    # Print metrics for current fold
    print(f"Metrics for Fold {fold}:")
    print(pd.DataFrame([metrics]))

# Summarize all folds metrics
all_metrics_df = pd.DataFrame(metrics_list)
print("\nSummary of Metrics Across All Folds (Cross-Validation on Training Set):")
print(all_metrics_df.mean())

# --- Final evaluation on the unseen test data (20% test set) ---
# Train the classifier on the entire training set (with SMOTE)
classifier.fit(X_train_smote, y_train_smote)

# Predict on the test set
y_pred_test = classifier.predict(X_test_scaled)

# Calculate metrics on the test set
accuracy_test = accuracy_score(y_test, y_pred_test)
report_test = classification_report(y_test, y_pred_test, target_names=['High', 'Low'])

# Print the final metrics for the test set
print(f"\nAccuracy on test data (unseen 20%): {accuracy_test}")
print("Classification Report on test data:")
print(report_test)


Test data



In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report, accuracy_score
from sklearn.preprocessing import MinMaxScaler

# Load your data
file_path = '/content/drive/MyDrive/Research_Dysgraphia/ANN_modified_data.csv'
data = pd.read_csv(file_path)
X = data.drop('severity', axis=1)  # Features
y = data['severity']  # Target variable

# Split the data into training (80%) and testing (20%) sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Normalize features
scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)  # Important to use the same scaler for the test set

# Apply SMOTE to the training set (not the test set)
smote = SMOTE(sampling_strategy='auto', random_state=42)
X_train_smote, y_train_smote = smote.fit_resample(X_train_scaled, y_train)

# Train the MLPClassifier on the SMOTE-applied training set
classifier = MLPClassifier(hidden_layer_sizes=(100, 50), max_iter=300, activation='relu', random_state=42)
classifier.fit(X_train_smote, y_train_smote)

# Predict on the test data (20% unseen data)
y_pred_test = classifier.predict(X_test_scaled)

# Calculate accuracy and other metrics on the test data
accuracy_test = accuracy_score(y_test, y_pred_test)
report_test = classification_report(y_test, y_pred_test, target_names=['High', 'Low'])

# Output the results for the test set
print(f"Accuracy on the test data (unseen 20%): {accuracy_test}")
print("Classification Report on the test data:")
print(report_test)


In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import StratifiedKFold, GridSearchCV
from imblearn.over_sampling import ADASYN
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score
from sklearn.preprocessing import MinMaxScaler

# Load your data
file_path = '/content/drive/MyDrive/Research_Dysgraphia/ANN_modified_data.csv'
data = pd.read_csv(file_path)
X = data.drop('severity', axis=1)
y = data['severity']

# Normalize features
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)

# Define cross-validator and hyperparameter grid for RandomForestClassifier
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Define the hyperparameter grid for Random Forest
param_grid = {
    'n_estimators': [100, 200, 300],  # Number of trees
    'max_depth': [10, 20, 30, None],  # Maximum depth of each tree
    'min_samples_split': [2, 5, 10],  # Minimum number of samples required to split an internal node
    'min_samples_leaf': [1, 2, 4],    # Minimum number of samples required to be at a leaf node
    'bootstrap': [True, False]        # Whether bootstrap samples are used when building trees
}

# Initialize the RandomForestClassifier
rf = RandomForestClassifier(random_state=42)

# Set up GridSearchCV
grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, cv=skf, scoring='accuracy', n_jobs=-1)

# Lists to store results
metrics_list = []

# Perform cross-validation
fold = 0
for train_index, test_index in skf.split(X_scaled, y):
    fold += 1
    X_train_fold, X_test_fold = X_scaled[train_index], X_scaled[test_index]
    y_train_fold, y_test_fold = y[train_index], y[test_index]

    # Apply ADASYN (SMOTE alternative) on each fold training data
    smote = ADASYN(sampling_strategy='auto', random_state=42)
    X_train_fold_smote, y_train_fold_smote = smote.fit_resample(X_train_fold, y_train_fold)

    # Perform grid search to find the best hyperparameters
    grid_search.fit(X_train_fold_smote, y_train_fold_smote)

    # Train the classifier with the best hyperparameters found
    best_classifier = grid_search.best_estimator_

    # Predict on the training fold (training accuracy)
    y_train_pred = best_classifier.predict(X_train_fold_smote)

    # Calculate training accuracy and other metrics
    accuracy_train = accuracy_score(y_train_fold_smote, y_train_pred)
    report_train = classification_report(y_train_fold_smote, y_train_pred, target_names=['High', 'Low'], output_dict=True)

    # Store metrics for each fold
    metrics = {
        'Fold': fold,
        'Training Accuracy': accuracy_train,
        'High Precision': report_train['High']['precision'],
        'High Recall': report_train['High']['recall'],
        'High F1': report_train['High']['f1-score'],
        'Low Precision': report_train['Low']['precision'],
        'Low Recall': report_train['Low']['recall'],
        'Low F1': report_train['Low']['f1-score'],
    }
    metrics_list.append(metrics)

    # Print training metrics for the current fold
    print(f"Training Metrics for Fold {fold}:")
    print(pd.DataFrame([metrics]))

    # Print the best hyperparameters for the current fold
    print(f"Best hyperparameters for Fold {fold}: {grid_search.best_params_}")

# Summarize training metrics across all folds
all_metrics_df = pd.DataFrame(metrics_list)
print("\nSummary of Training Metrics Across All Folds:")
print(all_metrics_df.mean())
